## LSH(Locality Sensitive Hashing)
LSH算法是一个被广泛使用的ANN算法，它可以在保证质量较高的搜索结果的同时，仍然保持高效的搜索速度。我们在这一章会阐述LSH的原理，同时用python实现一个自己的LSH搜索引擎。

In [ ]:
print('Welcome to LSH!')

### Prepare Data
如果你已经下载了数据，可以直接跳过这个cell

In [1]:
import shutil
import urllib.request as request
import tarfile
from contextlib import closing

# download the Sift1M dataset
with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/siftsmall.tar.gz')) as r:
    with open('sift.tar.gz', 'wb') as f:
        shutil.copyfileobj(r, f)


# the download leaves us with a tar.gz file, we unzip it
tar = tarfile.open('sift.tar.gz', "r:gz")
tar.extractall()

在讲LSH算法之前，我们先回忆一下我们平时接触到的HASH算法。他们有以下几个特点
1. 如果两个元素完全相同(每个bit都一样),那么会产生相同的hashcode。
2. 如果两个元素哪怕只有一个bit不同，最后产生的hashcode也会相差较大。

但是这样的HASH算法并不适合我们的ANN搜索。举一个例子，比如说我们有下面两篇文档
> I like dog

> I really like dog

如果我们使用传统的hash算法，即使这两篇文档只差了一个**really**，仍旧会产生完全不同的hashcode
| doc | MD5 | SHA1 |
| --- | --- | ---- |
|I like dog| 8a8249c0591b1c9aacc86f7a7e62fec3 | cf8ddb14b51c85d344cc7d841be5963b60a6d1a6 |
|I really like dog| 28316b7215c792aeea598f132e9e9541 | f541e674f949f39a5c1df53e45b46f0b5d518820 |

这就意味着如果我们基于hash来进行搜索,当输入为`I like dog`时,我们无法检索到`I really like dog`,这显然不是我们所期望的。而LSH通过巧妙的方法，让我们可以通过基于hash的方式，来搜索到相似的文档。

LSH的主要思想可以归纳为
1. 如果两篇文档相似，那么让他们的hashcode尽可能的相同，即发生碰撞。
2. 如果两篇文档不相似，那么尽可能的让他们hashcode不相同。

为此，我们需要有一个hash函数可以满足上述的两个要求。常见的hash函数有**MinHash**、**Random Projection**和**Cosine Similarity Hashing**等。本章的后续篇幅主要介绍**Random Projection**

### Random Projection
为了便于理解，我们从一个三维的例子出发.


<div style="text-align: center">
<img src="./resources/lsh_1.png"/>
</div>

假设我们现在向量空间中有六个点，我们可以任意选择一个超平面，这个超平面会将整个向量空间一分为2.

<div style="text-align: center">
<img src="./resources/lsh_2.png"/>
</div>

从图中我们可以看到，点A，B，C，D处于超平面的正面，而点E，F则处于超平面的负面。因此我们可以将A，B，C，D的hashcode设置为1, 点E，F的hashcode设置为0。这种hash方式符合LSH的要求，即相似的向量hashcode相同。

````
Hash(A) = Hash(B)
Hash(C) = Hash(D)
Hash(E) = Hash(F)
````
但是LSH的第二个要求并不满足，我们的`Hash(A) == Hash(C)`, 根据要求,我们应该有`Hash(A) != Hash(C)`。为了解决这个问题，我们可以再多加一个超平面.

<div style="text-align: center">
<img src="./resources/lsh_3.png"/>
</div>

通过增加超平面的方式，我们可以将hashcode从一个比特变成两个比特,也完成了LSH的hash函数要求。
````
Hash(A) = Hash(B) = 10
Hash(C) = Hash(D) = 11
Hash(A) = Hash(B) = 01

````
上述的方法就是随机投影法。我们通过随机生成固定数量N的超平面后，根据向量是处在超平面的正面还是负面，将向量hash为Nbit的hashcode。这一方法符合直觉，如果两个点相似，即距离相近，那么通过随机投影，他们会有较高的概率拥有相同的hashcode，但如果相距较远，他们的拥有相同的hashcode概率也会较低。同时计算一个点在超平面的正面还是负面也相当简单，你只需要对他们做点积即可。
````python
if np.dot(a, hyper_plane.T) > 0:
    return 1
return 0 

````
因此我们可以低成本的计算出向量的hashcode。
